In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import IPython
import matplotlib.pyplot as plt
import numpy as np
import soundfile as sf
import time
from tqdm import tqdm

from nara_wpe.wpe import online_wpe_step, get_power_online, OnlineWPE
from nara_wpe.utils import stft, istft, get_stft_center_frequencies
from nara_wpe import project_root

In [ ]:
stft_options = dict(size=512, shift=128)

# Example with real audio recordings
The iterations are dropped in contrast to the offline version. To use past observations the correlation matrix and the correlation vector are calculated recursively with a decaying window. $\alpha$ is the decay factor.

### Setup

In [ ]:
channels = 8
sampling_rate = 16000
delay = 3
alpha=0.9999
taps = 10
frequency_bins = stft_options['size'] // 2 + 1

### Audio data

In [ ]:
file_template = 'AMI_WSJ20-Array1-{}_T10c0201.wav'
signal_list = [
    sf.read(str(project_root / 'data' / file_template.format(d + 1)))[0]
    for d in range(channels)
]
y = np.stack(signal_list, axis=0)
IPython.display.Audio(y[0], rate=sampling_rate)

### Online buffer
For simplicity the STFT is performed before providing the frames.

Shape: (frames, frequency bins, channels)

frames: K+delay+1

In [ ]:
Y = stft(y, **stft_options).transpose(1, 2, 0)
T, _, _ = Y.shape

def aquire_framebuffer():
    buffer = list(Y[:taps+delay+1, :, :])
    for t in range(taps+delay+1, T):
        yield np.array(buffer)
        buffer.append(Y[t, :, :])
        buffer.pop(0)

### Non-iterative frame online approach
A frame online example requires, that certain state variables are kept from frame to frame. That is the inverse correlation matrix $\text{R}_{t, f}^{-1}$ which is stored in Q and initialized with an identity matrix, as well as filter coefficient matrix that is stored in G and initialized with zeros. 

Again for simplicity the ISTFT is applied afterwards.

In [ ]:
Z_list = []
Q = np.stack([np.identity(channels * taps) for a in range(frequency_bins)])
G = np.zeros((frequency_bins, channels * taps, channels))

for Y_step in tqdm(aquire_framebuffer()):
    Z, Q, G = online_wpe_step(
        Y_step,
        get_power_online(Y_step.transpose(1, 2, 0)),
        Q,
        G,
        alpha=alpha,
        taps=taps,
        delay=delay
    )
    Z_list.append(Z)

Z_stacked = np.stack(Z_list)
z = istft(np.asarray(Z_stacked).transpose(2, 0, 1), size=stft_options['size'], shift=stft_options['shift'])

IPython.display.Audio(z[0], rate=sampling_rate)

## Frame online WPE in class fashion:

Online WPE class holds the correlation Matrix and the coefficient matrix. 

In [ ]:
Z_list = []
online_wpe = OnlineWPE(
    taps=taps,
    delay=delay,
    alpha=alpha
)
for Y_step in tqdm(aquire_framebuffer()):
    Z_list.append(online_wpe.step_frame(Y_step))

Z = np.stack(Z_list)
z = istft(np.asarray(Z).transpose(2, 0, 1), size=stft_options['size'], shift=stft_options['shift'])

IPython.display.Audio(z[0], rate=sampling_rate)

# Power spectrum
Before and after applying WPE.

In [ ]:
fig, [ax1, ax2] = plt.subplots(1, 2, figsize=(20, 8))
im1 = ax1.imshow(20 * np.log10(np.abs(Y[200:400, :, 0])).T, origin='lower')
ax1.set_xlabel('')
_ = ax1.set_title('reverberated')
im2 = ax2.imshow(20 * np.log10(np.abs(Z_stacked[200:400, :, 0])).T, origin='lower')
_ = ax2.set_title('dereverberated')
cb = fig.colorbar(im1)